In [ ]:
%%writefile basicfunction.cpp

#include <iostream>
#include <iomanip>
#include <numeric>
#include <string>
#include <ctime>
#include <cstdlib>

using namespace std;

struct node{
    int value;
    node *next;
};//節點宣告

void insert_node(node * &node_in, int );
void dele_node(node * &node_in);
void card_use(node * &user_list, int** attacked);
void opcard_use(node * &oppo_list);
void Grid(int **g ,int,int);
void seaCreatures( int , int , int** g);
void Move(int **g,int[],int[],int,int,int,int);
void Seagull( int** s, int , int);
bool Gameover( int** g, int ,int );
void Sequense();

int main(){
    
    int row=0, column=0;
    string hit_res = "NULL";
    string win_res = "NULL";
    int card_res = 0; int card_num = 0; int card = 0;
    node * my_card = NULL;//存放抽卡結果
    node * oppo_card = NULL;

    cout<<"please input the size of your grid(2 numbers row and column): " ;
    cin >> row >> column;

    int grid[row+1][column+1]={};
    int attacked[row+1][column+1]={};
    int seagull[row+1][column+1]={};

    int* g[row+1];
    int* a[row+1];//a是一個row+1格指標陣列
    int* s[row+1];

    //將grid輸入到*g，row指向grid的每一格中一維陣列
    for(int j=0; j<row+1 ; ++j){
        g[j] = grid[j];
        a[j] = attacked[j];
        s[j] = seagull[j];
    }

    //place seagull and seacreatures if player choose Elite mode
    //不做防呆 選其他數字就是basic mode
    int mode=0;
    cout << "Game mode: 1.Basic mode 2.Elite mode 3.Funny mode" << endl;
    cin >> mode;
    if (mode==2){
      Seagull( s,row,column);
      seaCreatures( row,column, g);
    }
    else if (mode==3){
      seaCreatures( row,column, g);  
    }

    cout <<"your grid: "<< endl;
    Grid(g,row,column);
                                        //放船
    int ship_num = 1;
    cout << "Input the number of ship: "; 
    cin >> ship_num;

    int ship_length_c, ship_length_r, ship_size, ship_size_move=0; //船的長寬、船的總面積(佔幾格)
    int r=0, c=0; //船的起始位置(r,c)
    int setR[20]={}, setC[20]={};

    for(int d = 1; d <= ship_num; d++){
        //輸入船的長寬 Ship Size
        cout << "Input the length_column of your ship " << d << ": ";
        while(cin >> ship_length_c){
          if( ship_length_c <= column )break;
          cout << "Input the length_column of your ship " << d << ": ";
        }
        cout << "Input the length_row of your ship " << d << ": " ;
        cin >> ship_length_r;
        if(ship_length_r > row){
          cout << "The ship length_row is larger than game size, the length should shorter than " << row << endl;
          cout << "Input the length_row of your ship " << d << ": " ;
          cin >> ship_length_r;
        }
        
        ship_size = ship_length_c*ship_length_r;
        cout << "\n** The ship size is: " << ship_size << endl ;
        if (ship_size <= 5){
          cout << "** Your ship is a canoe!" << endl ;
        }
        else if (ship_size > 5 && ship_size < 10){
          cout << "** Your ship is a submarine!" << endl;
        }
        else
          cout << "** Your ship is an aircraft carrier!" << endl;



        //船的位置 Ship Position

        cout << "\nInput the beginning position of your ship (2 numbers row and column): " ; //以座標(r,c)作為船最左上的起始位置
        cin >> r >> c;
        cout << "You want to put the ship at: (" << r << "," << c << ")" <<endl ;

        while(r<1||c<1){  //防呆：數字需大於1
          cout << "Warning! The ship should be put in the position｜r:1-"<< row << " ，c:1-" << column << endl;
          cin >> r >> c;
        }

        int ship_and_wall_spacing_c = c + ship_length_c - column-1;  //>0:船超出牆的界線 <=0:船在界線內
        int ship_and_wall_spacing_r = r + ship_length_r - row-1;  //>0:船超出牆的界線 <=0:船在界線內

        if( ship_and_wall_spacing_c > 0 ){ //如果船的x邊長度超出牆
          cout << "The beginning position automatically moves to the left for " 
            << ship_and_wall_spacing_c << " grids since it exceeds the game size. " << endl;
          c -= ship_and_wall_spacing_c;
        }
        
        if( ship_and_wall_spacing_r > 0 ){ //如果船的y邊長度超出牆
          cout << "The beginning position automatically moves lower for " 
            << ship_and_wall_spacing_r << " grids since it exceeds the game size. " << endl;
          r -= ship_and_wall_spacing_r;
        }
          cout << "** The beginning position is at: (" << r << "," << c << ")" <<endl <<endl ;

        if ( d==2 ){ 
            ship_size_move = ship_length_c*ship_length_r;
            if ( ship_length_r == 1 || ship_length_c == 1 ){ //只允許 1*隨機數的船隻
              setR[0]=r;
              setC[0]=c;
              for(int ship_block=1 ; ship_block < ship_length_r ; ship_block++ ){
                if (ship_length_r>1){               //length_r長 setR的數字動 setC的數字不動
                  setR[ship_block] = r + ship_block;
                  setC[ship_block] = c;
                }
                else break;
              }
              for(int ship_block=1 ; ship_block < ship_length_c ; ship_block++ ){
                if (ship_length_c>1){               //length_c長 setC的數字動 setR的數字不動
                  setC[ship_block] = c + ship_block;
                  setR[ship_block] = r;
                }
                else break;
              }
            }
        }
        for (int a = r  ; a < r + ship_length_r ; a++ )// 假設矩陣的row變數為為row
        {
            for (int b = c ; b < c + ship_length_c ; b++)//假設矩陣的column變數為column
            {
                  grid[a][b] = 1;
            }
          }
    }

     // display grid
    Grid(g,row,column);
    int round=0;

    int Whofirst=0;
    while( Whofirst!=1 && Whofirst!=2 ){
      cout << "Who goes first? 1.We 2.Opponent." << endl;
      cin >> Whofirst;
    }

    cout <<"---------------First round---------------" << endl;

    while( Gameover( g, row ,column ) ){
          
          //抽卡
          if (mode==2){
              card = 1 + rand()%6;//產生產生1~5的亂數
              insert_node(my_card, card);//儲存自己的卡

              card = 1 + rand()%6;//產生產生1~5的亂數
              insert_node(oppo_card, card);//儲存對手的卡

              cout << "********** Draw a card automatically **********" << endl;


              //使用卡
              cout << "Would you like to use your magical card? (1: yes, 0: no)" << endl;
              cin >> card_res;

                  if(card_res == 1){   //這裡對嗎? 之前少了一個}把他補上去了
                  card_use(my_card, a);

                  }
              cout << "Would your opponents like to use their magical card? (1: yes, 0: no)" << endl;
              cin >> card_res;
              if(card_res == 1)
                  opcard_use(oppo_card);
          }

          //<<Opponent攻擊first>>
      
          if (Whofirst==2){
              int attackR=0, attackC =0;

              cout << "\nWhere do your opponents want to attack? Please input the position:" <<endl;
              cin >> attackR >> attackC;

              while( attackR < 1 || attackR > row || attackC < 1 || attackC > column ){
                  cout << "Where do your opponents want to attack? Please input the position:" <<endl;
                  cin >> attackR >> attackC;
              }
            
              if( seagull[attackR][attackC] != 1 ){   // 確認攻擊情況(攻擊到海鷗會使該次攻擊無效化，並告訴攻擊方擊中未知目標，可能是船也可能是海鷗)
              
                if (grid[attackR][attackC] == 1){ // 假設船的預設會是1
                        cout << "Oh damn, they shot it." << endl;
                        grid[attackR][attackC] = 8;
                    }
                    else if( grid[attackR][attackC] == 2 ){  //當擊中位置為2時
                    cout << "You hit the whale!! Say sorry to the whale!!" << endl;
                    }
                    else  if( grid[attackR][attackC] == 3 ){
                    cout << "You hit a jellyfish" << endl;
                    }
                    else 
                    cout << "Missed! They're stupid! lol" <<endl;
                }
                else {
                  cout << "You hit the seagull in the sky, and it makes your attack invalid!" << endl;
                  seagull[attackR][attackC]=0;
                }
                    
              // Display grid
              Grid(g,row,column);
              
              cout << "Where do you want to attack? Please input the position:" <<endl;
              cin >> attackR >> attackC;
              while( attackR < 1 || attackR > row || attackC < 1 || attackC > column ){
                  cout << "Where do you want to attack? Please input the position:" <<endl;
                  cin >> attackR >> attackC;
              }
              cout << "Please type in hitting result(hit or miss): ";
              cin >> hit_res;
              
              if(hit_res == "hit")
                  attacked[attackR][attackC] =8;
              else if(hit_res == "miss")
                  attacked[attackR][attackC] =1;
              else{
                  while(1){
                      cout << "Please type in the hitting result(hit or miss): ";
                      cin >> hit_res;
                      if(hit_res == "hit" or hit_res == "miss")
                          break;
                  }
              }
              cout << "\n" << "*********** attack record ******************" << endl;
              Grid(a,row,column);
              cout << "8: hit" << "\n" << "1: miss" << "\n" << "0: not yet hit" << endl;
              cout << "*********** end of attack record ***********" << endl;

          }
              
          if(Whofirst==1){     
              // **Hitting Position Entry & detection**
              int attackR=0, attackC =0;
              cout << "\nWhere do you want to attack? Please input the position:" <<endl;
              cin >> attackR >> attackC;
              //防呆，不能輸入超過或小於
              while( attackR < 1 || attackR > row || attackC < 1 || attackC > column ){
                  cout << "Where do you want to attack? Please input the position:" <<endl;
                  cin >> attackR >> attackC;
              }
              cout << "Please type in hitting result(hit or miss): ";
              cin >> hit_res;
              
              if(hit_res == "hit")
                  attacked[attackR][attackC] =8;
              else if(hit_res == "miss")
                  attacked[attackR][attackC] =1;
              else{
                  while(1){
                      cout << "Please type in the hitting result(hit or miss): ";
                      cin >> hit_res;
                      if(hit_res == "hit" or hit_res == "miss")
                          break;
                  }
              }
              
              cout << "\n" << "*********** attack record ******************" << endl;
              Grid(a,row,column);
              cout << "8: hit" << "\n" << "1: miss" << "\n" << "0: not yet hit" << endl;
              cout << "*********** end of attack record ***********" << endl;
              
              //<<Opponent攻擊>>


              cout << "Where do your opponents want to attack? Please input the position:" <<endl;
              cin >> attackR >> attackC;
              //防呆，不能輸入大於棋盤或小於1
              while( attackR < 1 || attackR > row || attackC < 1 || attackC > column ){
                  cout << "Where do your opponents want to attack? Please input the position:" <<endl;
                  cin >> attackR >> attackC;
              }
            
              if( seagull[attackR][attackC] != 1 ){   // 確認攻擊情況(攻擊到海鷗會使該次攻擊無效化，並告訴攻擊方擊中未知目標，可能是船也可能是海鷗)
              
                if (grid[attackR][attackC] == 1){ // 假設船的預設會是1
                        cout << "Oh damn, they shot it." << endl;
                        grid[attackR][attackC] = 8;
                    }
                    else if( grid[attackR][attackC] == 2 ){  //當擊中位置為2時
                    cout << "You hit the whale!! Say sorry to the whale!!" << endl;
                    }
                    else  if( grid[attackR][attackC] == 3 ){
                    cout << "You hit a jellyfish" << endl;
                    }
                    else 
                    cout << "Missed! They're stupid! lol" <<endl;
                }
                else {
                  cout << "You hit the seagull in the sky, and it makes your attack invalid!" << endl;
                  seagull[attackR][attackC]=0;
                }
                // Display grid
                Grid(g,row,column);

          }
          
          //船隨機移動，兩艘船才能用
          int Direction;
          if(mode==2||mode==3){
            Direction = rand()%(ship_size_move*5);
            Move( g,setR,setC,row,column,ship_size_move,Direction);
            if(Direction==1||Direction==2||Direction==3||Direction==4)cout << "\n浪來了~~" << endl;
          } 
           
          cout << "Input your opponents' response(win or continue)";
          cin >> win_res;
          if(win_res == "win"){
              cout <<"\nGAME OVER! You win!"; //對方擊中我們所有的船
              break;
          }
          cout <<"---------------Next round---------------" << endl;
          round++;
          
          
    }
    if(win_res != "win"){
        cout << "\nGAME OVER! You Lose!";
    }

     // Display rounds
      cout << "You have played for " << round << " rounds" << endl;
      return 0;
}


void Grid(int **g ,int row ,int column){   
      // <<Ｘ軸座標標示>>
    cout << "\t";
      for (int i = 1 ; i <= column ; i++){
          cout << i <<"\t";
      }
      cout << "C\n";

      for (int i=1 ; i <= row ; i++){
        cout << i <<"\t" ;   //<<Ｙ軸座標標示>>
          for (int j = 1 ; j <= column ; j++){
             cout <<  g[i][j] <<"\t";
          }
          cout <<"\n";
      }
      cout<<"R"<<endl;  // <<Ｘ軸座標標示>>
}

void seaCreatures( int row, int column, int** g){
    
  srand(time(0));

  int whaleX = 1+rand()%row, whaleY = 1+rand()%column;    //在海中佈置鯨魚(海底生物1)，共一隻鯨魚，面積為1x2
  int whale_and_wall_spacing_x = whaleX + 1 - column - 1;
  int whale_and_wall_spacing_y = whaleY + 2 - column - 1;

  if( whale_and_wall_spacing_x > 0 ){ // 如果鯨魚的x邊長度超出牆
    whaleX -= whale_and_wall_spacing_x;
  }

  if( whale_and_wall_spacing_y > 0 ){ // 如果鯨魚的y邊長度超出牆                               
    whaleY -= whale_and_wall_spacing_y;
  }

  for(int a = whaleX; a < whaleX + 1; a++){ 
    for(int b = whaleY; b < whaleY + 2; b++){ 
      if( g[a][b] != 1){   //將鯨魚位置設置在非船位置上，若擊中則告訴攻擊方擊中鯨魚
        g[a][b] = 2;    //將海底生物位置設為2
      }
    }
  }

  for(int i=0; i<5; i++){   //在海中佈置水母(海底生物2)，共30隻，面積為1x1
      
    int jellyfishX = 1+rand()%row;
    int jellyfishY = 1+rand()%column;

    if( g[jellyfishX][jellyfishY] != 1){
      g[jellyfishX][jellyfishY] = 3;
    }
  }
}

void Move(int **g,int setR[],int setC[],int row,int column,int shipsize,int Direction){  
  bool collusion=false;
  if(g[setR[0]][setC[0]]==1 && g[setR[1]][setC[1]]==1) { 
    switch(Direction%5){
        case 1 : //上移
          
          if(setR[0]==1)break; //碰到上邊，保持不動
          for(int j=0; j<shipsize ; j++ ){
            for(int c=0; c<shipsize ;c++){
              if(g[setR[j]-1][setC[j]+c]==1)collusion=true; //碰到船，停
            }
          }
          if(collusion)break;
          for(int i=0 ; i<shipsize ; i++){ //將原本位置歸零
              g[setR[i]][setC[i]]=0;                    
          } 
          for(int j=0 ; j<shipsize ; j++){ //填入新的位置
              setR[j]=setR[j]-1;
              setC[j]=setC[j];                      
              g[setR[j]][setC[j]]=1;                    
          }  
          break; //下移
        case 2 :
          if(setR[shipsize-1]==row)break;
          for(int j=0; j<shipsize ; j++ ){
            for(int c=0; c<shipsize ;c++){
              if(g[setR[j]+1][setC[j]+c]==1)collusion=true; //碰到船，停
            }
          }
          if(collusion)break;
          for(int i=0 ; i<shipsize ; i++){ //將原本位置歸零
             g[setR[i]][setC[i]]=0;                    
          } 
          for(int j=0 ; j<shipsize ; j++){ //
             setR[j]=setR[j]+1;
             setC[j]=setC[j];                      
             g[setR[j]][setC[j]]=1;                    
          }  
          break;
        case 3 : //左移
          if(setC[0]==1)break; //碰到左邊
          for(int j=0; j<shipsize ; j++ ){
            for(int r=0; r<shipsize ;r++){
              if(g[setR[j]+r][setC[j]-1]==1)collusion=true; //碰到船，停
            }
          }
          if(collusion)break;
          for(int i=0 ; i<shipsize ; i++){ //與shipsize有關
             g[setR[i]][setC[i]]=0;                    
          } 
          for(int j=0 ; j<shipsize ; j++){ //與shipsize有關
             setR[j]=setR[j];
             setC[j]=setC[j]-1;                      
             g[setR[j]][setC[j]]=1;                    
          }  
          break; 
        case 4 ://右移
          if(setC[shipsize-1]==column)break;
          for(int j=0; j<shipsize ; j++ ){
            for(int r=0; r<shipsize ;r++){
              if(g[setR[j]+r][setC[j]+1]==1)collusion=true; //碰到船，停
            }
          }
          if(collusion)break;
          for(int i=0 ; i<shipsize ; i++){ //與shipsize有關
             g[setR[i]][setC[i]]=0;                    
          } 
          for(int j=0 ; j<shipsize ; j++){ //與shipsize有關
             setR[j]=setR[j];
             setC[j]=setC[j]+1;                      
             g[setR[j]][setC[j]]=1;                    
          }  
          break;
        default: break;   
    }
  }
}

void Seagull( int** s,  int row, int column){
  srand(time(0));
  int m, n;

  for(int k = 0; k < 8; k++){  //在空中隨機佈置海鷗位置，共8隻，體積為1x1    
    m = 1+rand()%row;
    n = 1+rand()%column;
    s[m][n] = 1;
  }
}


void insert_node(node * &node_in, int card){
    node * t = new node();
    t->value = card;
    t->next = 0;

    if(node_in == NULL)
      node_in = t;
    else
      node_in->next = t;

}

void dele_node(node * &node_in){
    if( node_in == NULL){
        return;
    }
    else{
        node * t = node_in;
        node_in = node_in -> next;
        delete t;
    }
}

void card_use(node * &my_list, int** attacked){
    int row; int column;
    string hit_res; string pos_res;
     switch(my_list->value){
          case 1: 
              {cout << " Miss a turn " << endl; break;}
          case 2: 
              {cout << " The other side misses a turn " << endl; break;}
          case 3: 
              {cout << " Ask the other side to tell you the item in the position " << endl << "Input position (2 numbers for row and column): " << endl; 
                cin >> row >> column;
                cout << "Input hit or miss: ";
                cin >> hit_res;
                if(pos_res == "hit")
                  attacked[row][column] = 8;
                else{
                  attacked[row][column] = 0;
                }
                break;}
          case 4:
              {cout << " Tell the other side the position info " << endl; break;}
    }

    dele_node(my_list);

}

void opcard_use(node * &oppo_list){
     switch(oppo_list->value){
          case 1: 
              {cout << "Miss a turn " << endl; break;}
          case 2: 
              {cout << "The other side miss a turn " << endl; break;}
          case 3: 
              {cout << "Ask the other side to tell you the item in the position " << endl;
                break;}
          case 4:
              {cout << "Tell the other side the position info " << endl; break;}
          case 5:
              {cout << "印出冷笑話 " << endl; break;}
    }

    dele_node(oppo_list);

}

bool Gameover(int** g , int row , int column){
      int countOne = 0;
          for (int m = 1 ; m <= row ; m++)// 假設矩陣的row變數為為row
          {
              for (int n  = 1 ; n <= column ; n++)//假設矩陣的column變數為column
              {
                  if ( g[m][n] != 1 ) //沒有1就是就是gameover
                      countOne++;//計算不是1的個數
              }
          }
          if (countOne == (row)*(column)){   //對方擊中我們所有的船
              return false;
          }
          else  return true;
}
void Sequense()
{
    int turn;
    srand(time(0));

    turn = rand() % 2 + 1;

    if (turn == 1)
    {
        cout << "player 1 turn" << endl; //自己
    }
    else
    {
        cout << "player 2 turn" << endl; //敵人
    }
}

Overwriting basicfunction.cpp


In [ ]:
%%shell
g++ basicfunction.cpp -o basicfunction
./basicfunction

please input the size of your grid(2 numbers row and column): 4 4
Game mode: 1.Basic mode 2.Elite mode 3.Funny mode
3
your grid: 
	1	2	3	4	C
1	0	0	0	3	
2	3	3	0	0	
3	3	0	0	0	
4	0	0	3	2	
R
Input the number of ship: 2
Input the length_column of your ship 1: 1 1
Input the length_row of your ship 1: 
** The ship size is: 1
** Your ship is a canoe!

Input the beginning position of your ship (2 numbers row and column): 1 1
You want to put the ship at: (1,1)
** The beginning position is at: (1,1)

Input the length_column of your ship 2: 1 2
Input the length_row of your ship 2: 
** The ship size is: 2
** Your ship is a canoe!

Input the beginning position of your ship (2 numbers row and column): 3 3
You want to put the ship at: (3,3)
** The beginning position is at: (3,3)

	1	2	3	4	C
1	1	0	0	3	
2	3	3	0	0	
3	3	0	1	0	
4	0	0	1	2	
R
Who goes first? 1.We 2.Opponent.
1
---------------First round---------------

Where do you want to attack? Please input the position:
1 1
Please type in hitting result(

CalledProcessError: ignored